# Chapter 08: Tree

## 8.1 General Trees

### 8.1.1 Tree Definitions and Properties

A **tree** is an abstract data type that stores elements hierarchcially.

##### Formal Tree Definition

Formally, we define **tree** $T$ as a set of **nodes** storing elements such that the nodes have a **parent-child** relationship that satisfies the following properties:

* If $T$ is nonempty, it has a special node, called the **root** of $T$, that has no parent.
* Each $v$ of $T$ different from the root has a unique **parent** node $w$; every node with parent $w$ is a **child** of $w$.

##### Other Node Relationships

Two nodes that are children of the same parent are **siblings**. A node $v$ is **external** if $v$ has no children. A node $v$ is **internal** if it has one or more children. External nodes are also known as **leaves**.

##### Ordered Trees

A tree is **ordered** if there is a meaningful linear order among the children of each nodes; that is, we purposefully identify the children of a node as being the first, second, third, and so on.

### 8.1.2 The Tree Abstract Data Type

We define a tree ADT using the concept of a **position** as an abstraction for a node of a tree. An element is stored at each position, and positions satisfy parent-child relationships that define the tree structure. A position object for a tree supports the method:

* `p.element()`: Return the element stored at position `p`.
* `T.root()`: Return the position of the root of tree `T`, or `None` if `T`
* `T.is_root(p)`: Return `True` if position `p`is the root of Tree `T`.
* `T.parent(p)`: Return the position of the parent of position `p` or `None` if `p` is the root of `T`.
* `T.num_children(p)`: Return the number of children of position `p`
* `T.children(p)`: Generate an iteration of the children of position `p`.
* `T.is_leaf(p)`: Return `True` if position `p` does not have any children.
* `len(T)`: Return the number of positions (and hence elements) that are contained in tree `T`.
* `T.is_empty()`: Return `True` if tree `T` does not contain any positions.
* `T.positions()`: Generate an iteration of all positions of tree `T`.
* `iter(T)`: Generate an iteration of all elements stored within tree `T`.

In [1]:
from abc import ABC, abstractmethod

class Tree(ABC):
    """Abstract base class representing a tree structure."""
    
    class Position(ABC):
        """An abstraction representing the location of a single element."""
        
        @abstractmethod
        def element(self):
            """Return the element stored at this Position."""
            pass
        
        @abstractmethod
        def __eq__(self, other):
            """Return True if other Position represents the same location."""
            pass
        
        def __ne__(self, other):
            """Return True if other does not represent the same location."""
            return not (self == other)

    @abstractmethod
    def root(self):
        """Return Position representing the tree's root (or None if empty)."""
        pass
    
    @abstractmethod
    def parent(self, p):
        """Return Position representing p's parent (or None if p is root)."""
        pass
    
    @abstractmethod
    def num_children(self, p):
        """Return the number of children that Position p has."""
        pass
    
    @abstractmethod
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        pass
    
    @abstractmethod
    def __len__(self):
        """Return the total number of elements in the tree."""
        pass
    
    def is_root(self, p):
        """Return True if Position p represents the root of the tree."""
        return self.root() == p
    
    def is_leaf(self, p):
        """Return True if Position p does not have any children."""
        return self.num_children(p) == 0
    
    def is_empty(self):
        """Return True if the tree is empty."""
        return len(self) == 0
    
    def depth(self, p):
        if self.is_root(p):
            return 0
        else:
            return 1 + self.depth(self.parent(p))
        
    def _height2(self, p):
        if self.is_leaf(p):
            return 0
        else:
            return 1 + max(self._hegiht2(c) for c in self.children(p))
    
    def height(self, p=None):
        if p is None:
            p = self.root()
        return self._height2(p)

### 8.1.3 Computing Depth and Height

##### Depth

The **depth** of $p$ si the number of ancestor of $p$, excluding $p$ itself.

The running time of `T.depth(p)` for position `p` is $O(d_p + 1)$, where $d_p$ denotes the depth of $p$ in the tree $T$, because the algorithm performas a constant-time recursive step for each ancestor of $p$. Thus algorithm `T.depth(p)` runs in $O(n)$ worst-case time, where $n$ is the total number of positions of $T$, because a position of $T$ may have depth $n-1$ if all nodes from a single branch.

In [2]:
def depth(self, p):
    if self.is_root(p):
        return 0
    else:
        return 1 + self.depth(self.parent(p))

##### Height

The **height** of a position $p$ in a tree $T$ is also defined recursively:

* If $p$ is a fleaf, then the height of $p$ is 0
* Otherwise, the height of $p$ is one more thatn the maximum of the heights of $p$'s children.

*The height of a nonempty tree $T$ is equal to the maximum of the dpeths of its leaf positions.*

In [3]:
def _height1(self, p):
    return max(self.depth(p) for p in self.positions() if self.is_leaf(p))

However, algorithm `_hegiht1` is not very efficient. Because `_height1` calls algorithm `depth(p)` on each leaf of $T$, its running time is $O(n + \sum_{p \in L}(d_p + 1))$, where $L$ is the set of leaf positions of $T$. In the worst case, this will lead to $O(n^2)$.

It can be improved by:

In [4]:
def _height2(self, p):
    if self.is_leaf(p):
        return 0
    else:
        return 1 + max(self._hegiht2(c) for c in self.children(p))

It is important to understand why algorithm `height2` is more efficient than `height1`. The algorithm is recursive, and it progresses in a top-down fashion. If the method is initially called on the root of $T$, it will eventually be called once for each position of $T$. This is because the root eventually invokes the recursion on each of its children, which in turn invokes the recursion on each of their children, and so on.

We can determine the running time of the `height2` algorihtm by summing, over all the positions, the amount of time spent on the nonrecursive part of each call. In our implementation, there is a constant amound otf work per position, plus the overhead of computing the maximum over the iteration of children. Although we do not yet have a concrete implementation of `children(p)`, we assume that such an iteration is generated in $O(c_p + 1)$ time, where $c_p$ denotes the number of children
of $p$. Algorithm `height2` spends $O(c_p + 1)$ time at each position $p$ to compute the maximum, and its overall running time is $O(\sum_p (c_p + 1)) = O(n + \sum_p c_p)$.

*Let $T$ be a tree with $n$ positions, and let $c_p$ denote the number of children of a position $p$ of $T$. Then, summing over the positions of $T$, $\sum_p c_p = n-1$.*

By this proposition, the running time of algorithm `height2` is $O(n)$, where $n$ is the number of positions of $T$.

## 8.2 Binary Trees

A **binary tree** is an ordered tree with the following properties:

1. Every node has at most two children.
2. Each child node is alabeled as being either a left child or right child.
3. A left child precedes a right child in the order of children of a node.

The subtre rooted at a left or right child or an internal node $v$ is called a **left subtree** or **right subtree**. respectively, of $v$. A binary tree is **proper** if each node has either zero or two children. Some people also refer to such trees as being **full** binary trees. A binary tree that is not proper is **improper**.

### 8.2.1 The Binary Tree Abstract Data Type

* `T.left(p)`: Return the position that represents the left child of `p`, or `None` if `p` has no left child.
* `T.right(p)`: Return the position that represents the right child of `p` or `None` if `p` has no right child.
* `T.sibling(p)`: Return the position that represents the sibling of `p`, or `None` if `p` has no sibling.

In [5]:
class BinaryTree(Tree):
    """Abstract base class representing a binary tree structure."""
    
    @abstractmethod
    def left(self, p):
        """Return a Position representing p's left child.
        
        Return None if p does not have a left child.
        """
        pass
    
    @abstractmethod
    def right(self, p):
        """Return a Position representing p's right child.
        
        Return None if p does not have a right child.
        """
        pass
    
    def sibling(self, p):
        """Return a Position representing p's sibling (or None if no sibling)."""
        parent = self.parent(p)
        if parent is None:
            return None
        else:
            if p == self.left(parent):
                return self.right(parent)
            else:
                return self.left(parent)
            
    def children(self, p):
        """Generate an iteration of Positions representing p's children."""
        if self.left(p) is not None:
            yield self.left(p)
        
        if self.right(p) is not None:
            yield self.right(p)

### 8.2.2 Properties of Binary Trees

##### Proposition

Let $T$ be a nonempty binary tree, and let $n$, $n_W$, $n_I$ and $h$ denote the number of nodes, umber of external nodes, number of internal nodes, and height of $T$, respectively.Then $T$ has the following properties:

1. $h+1 \leq n \leq 2^{h+1} -1$
2. $1 \leq n_E \leq 2^h$
3. $h \leq n_I \leq 2^h - 1$
4. $\log(n+1) -1 \leq h \leq n - 1$

Also, if $T$ is proper, then $T$ has the following properties:

1. $2h + 1 \leq n \leq 2^{h+1} - 1$
2. $h +1 \leq n_E \leq 2^h$
3. $h \leq n_I \leq 2^h -1$
4. $\log(n+1) -1 \leq h \leq (n-1)/2$

##### Proposition

In a nonempty proper binary tree $T$, with $n_E$ external nodes and $n_I$ internal nodes, we have $n_E = n_I + 1$.

## 8.3 Implementing Trees

The `Tree` and `BinaryTree` classes defined so far are both formally **abstract base classes**. Especially, concrete implementations of `root`, `parent`, `num_children`, `children` `__len__`, `left` and `right` are not declared yet.

We begin with the case of **binary tree**, sicne its shape is more narrowly defined.

In [6]:
class LinkedBinaryTree(BinaryTree):
    """Linked representation of a binary tree structure."""

    class _Node:
        __slots__ = '_elements', '_parent', '_left', '_right'

        def __init__(self, element, parent=None, left=None, right=None):
            self._element = element
            self._parent = parent
            self._left = left
            self._right = right

    
    class Position(BinaryTree.Position):
        """An abstraction representing the location of a single element."""

        def __init__(self, container, node):
            """Constructor should not be invoked by user."""
            self._container = container
            self._node = node

        def element(self):
            return self._node._element

        
        def __eq__(self, other):
            return type(other) is type(self) and other._node is self._node

        def _validate(self, p):
            """Return associated node, if position is valid."""
            if not isinstance(p, self.Position):
                raise TypeError('p must be proper Position type')

            if p._container is not self:
                raise ValueError('p does not belong to this container')

            if p._node._parent is p._node:
                raise ValueError('p is no longer valid')
                return p._node
            return p._node

        def _make_position(self, node):
            """Return Position instance for given node (or None if no node)."""
            return self.Position(self, node) if node is not None else None


    def __init__(self):
        """Create an intially empty binary tree."""
        self._root = None
        self._size = 0
    

    def __len__(self):
        """Return the total number of elements in the tree."""
        return self._size

    def root(self):
        """Return the root Position of the tree (or None if tree is empty)."""
        return self._make_position(self._root)

    def parent(self, p):
        """Return the Position of p's parent(or None if p is root)."""
        node = self._validate(p)
        return self._make_position(node._parent)


    def left(self, p):
        """Return the Position of p's left child(or NOne if no left child)."""
        node = self._validate(p)
        return self._make_position(node._left)

    def right(self, p):
        """Return the Position of p's left child(or NOne if no left child)."""
        node = self._validate(p)
        return self._make_position(node._right)

    def num_children(self, p):
        """Return the number of children of Position p."""
        node = self._validate(p)
        count = 0
        if node._left is not None:
            count += 1
        if node._right is not None:
            count += 1
        return count

    def _add_root(self, e):
        """Place element e at the root of an empty tree and return new Position.

        Raise ValueError if tree nonempty.
        """

        if self._root is not None: raise ValueError('Root exists')
        self._size = 1
        self._root = self._Node(e)
        return self._make_position(self._root)

    def _add_left(self, p, e):
        """Create a new left child for Position p, storing elment e.
        
        Return the Position of new node
        Raise ValueError if Position p is invalidor p already has a left child.
        """

        node = self._validate(p)
        if node._right is not None: raiseValueError('Left child exists')
        self._size += 1
        node._left = self._Node(e, node)
        return self._make_position(node._left)

    def _add_right(self, p, e):
        """Create a new right child for Position p, storing element e..git/
        
        Return the Position of new node
        Raise ValueError if Position p is invalid or p already has a right child.
        """

        node = self._validate(p)
        if node._rihgt is not None: raise ValueError('Right child exists')
        self._size += 1
        node._right = self._Node(e, node)
        return self._make_position(node._right)


    def _replace(self, p, e):
        """Replace the element at position p with e, and return old element."""
        node = self._validate(p)
        old = node._element
        node._element = e
        return old

    def _delete(self, p):
        """Delete the node at Position p, and replace it with its child, if any..git/

        Return the element that had been stored at Position p
        Raise ValueError if Position p is invalid or p has two children.
        """

        node = self._validate(p)
        if self.num_children(p) == 2: raise ValueError('p has two children')
        child = node._left if node._left else node._right
        if child is not None:
            child._parent = node._parent
        if node is self._root:
            self._root = child
        else:
            parent = node._parent
            if node is parent._left:
                parent._left = child
            else:
                parent._right = child
        self._size -= 1
        node._parent = node
        return node._element

    def _attach(self, p, t1, t2):
        """Attach trees t1 and t2 as left and right subtrees of external p."""
        node = self._validate(p)
        if not self.is_leaf(p): raise ValueError('position must be leaf')
        if not type(self) is type(t1) is type(t2):
            raise TypeError('Tree types must match')
        self._size += len(t1) + len(t2)
        if not t1.is_empty():
            t1._root._parent = node
            node._left = t1._root
            t1._root = None
            t1._size = 0
        if not t2.is_empty():
            t2._root._parent = node
            node._right = t2.root
            t2._root = None
            t2._size = 0


#### Performance of the Linked Binary Tree Implementation

|Operation|Running Time|
|---|---|
|`len`, `is_empty`| $O(1)$ |
|`root`, `parent`, `left`, `right`, `sibling`, `children`, `num_children` | $O(1)$ |
|`is_root`, `is_leaf` | $O(1)$ |
|`depth(p)` | $O(d_p + 1)$ |
|`height` | $O(n)$ |
|`add_root`, `add_left`, `add_right`, `replace`, `delete`, `attach` | $O(1)$ |

### 8.3.2 Array-Based Representation of a Binary Tree

Alternative way of representing binary tree T is based on a way of numbering the positions of T

* If $p$ is the root of $T$, then $f(p) = 0$.
* If $p$ is the left child of position $q$, then $f(p) = 2f(q) +1$
* If $p$ is the right child of position $q$, then $f(p) = 2f(q) + 2$

This type of numbering is known as a *level numbering*.

The level numbering function $f$ suggests a representation of a binary tree $T$ by means of an array-based structure $A$, with the element at position $p$ of $T$ stored at index $f(p)$ of the array.

For general binary trees, the exponential $O(2^n)$ worst-case space requirement of this representation is prohibitive.

Another drawback of level numbering is that some update operations for trees cannot be efficiently supported. (Deleting a node can affect all descendants)

### 8.3.3 Linked Structure for General Trees

For a general tree, there is no priori limit on the number of children that a node may have. Children can be set to a list and children of the node can refer the element of the list.

Summary of the performance of the implementation of a general tree using a linked structure.

|Operation|Running Time|
|---|---|
|`len`, `is_empty`|$O(1)$|
|`root`, `parent`, `is_root`, `is_leaf` |$O(1)$|
|`children(p)`|$O(c_p + 1)$|
|`depth(p)`|$O(d_p + 1)$|
|`height`|$O(n)$|

$c_p$ denotes the number of children of a position $p$

## 8.4 Tree Traversal Algorithms

A *traversal* of a tree $T$ is a systematic way of accessing, or "visiting," all the possible positions of $T$.

### 8.4.1 Preporder and Postorder Traversals of General Trees

#### Preorder traversal

```
Algorithm preorder(T, p):
  perform the "visit" action for position p
  for each child c in T.children(p) do
    preorder(T, c)
```

#### Postorder Traversal

```
Algorithm postorder(T, p):
  for each child c in T.children(p) do
    postorder(T, c)
perform the "visit" action for position p
```

#### Running-Time analysis

At each position $p$, the nonrecursive part of the traversal algorithm requires time $O(c_p + 1)$, where $c_p$ is the number of children of $p$, under the assumption that the "visit" itself takes $O(1)$ time.

The overall running time for the traversal of tree $T$ is $O(n)$, where $n$ is the nubmer of positions in the tree. This running time is asymptotically optimal since the traversal must visit all the $n$ positions of the tree.

### 8.4.2 Breadth-First Tree Traversal

The way of trasversing a tree that visits all the positions at depth $d$ before visit positions at depth $d + 1$ is called as **breadth-first traversal**

```
Algorithm breadth(T):
  Initialize queue Q to contain T.root()
  while Q not empty do
    p = Q.dequeue()
    perform the "visit" action for position p
    for each child c in T.children(p) do
      Q.enqueue(c)
```

### 8.4.3 Inorder Traversal of a Binary Tree

During an inorder traversal, we visit a position between the recursive traversals of its left and right subtrees. The inorder traversal of a binary tree $T$ can be informally viewecd as visiting the nodes of $T$ "from left to right." Indeed, for every position $p$, the inorder traversal visits $p$ after all the positions in the left subtree of $p$ and before all the positions in the right subtree of $p$.

```
Algorithm inorder(p):
  if p has a left child lc then
    inorder(lc)
perform the "visit" action for position p
if p has a right child rc then
  inorder(rc)
```

#### Binary Search Trees

An mportant application of the inorder traversal algorithm arises when we store an ordered sequnce of elements in a binary tree, defining a structure we call a binary search tree. Let $S$ be a set whose unique elements have an order relation. FOr example, $S$ could be a set of integers. A binary search tree for $S$ is a binary tree $T$ such that, for each position $p$ of $T$:

* Position $p$ stores an element of $S$, denoted as $e(p)$.
* Elements stored in  the left subtree of $p$ (if any) are less than $e(p)$.
* Elements stored in the right subtree of $p$ (if any) are greater than $e(p)$.

Running time of binary search tree is proportional to the height of $T$. It can be as small as $\log (n+1) -1$ or as large as $n -1$, when $n$ is the number of nodes of a tree. Thus, binary search trees are most efficient when they have small height.

![tree traversals](https://miro.medium.com/max/700/0*YzOEfnGnWTPbsUkv)

From: https://medium.com/@ajinkyajawale/inorder-preorder-postorder-traversal-of-binary-tree-58326119d8da

### 8.4.3 Implementing Tree Traversals in Python

In this context, `__iter__` method can be effectively used to implement Tree Traversals. The book suggests follwing baseline:

```python
def __iter__(self):
        """Generate an iteration of the tree's elements."""
        for p in self.positions():
            yield p.element()
        
```

`Positions` need to be implemented independently for each strategy.